In [1]:
import pandas as pd
from sqlalchemy import *
from create_ref_mariadb import mariadb_connection
from create_ref_mongodb import mongodb_connection
import pymongo
import pandas as pd
import pandas as pd
from sqlalchemy import *
from create_ref_mariadb import mariadb_connection
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import numpy as np
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import folium
import branca
import os
import vincent
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from io import BytesIO
import base64


In [2]:
#get data from ref
eng = mariadb_connection()
df = pd.read_sql(
    """SELECT windfarm_id,latitude,longitude
        FROM mariadb_itw.windfarms ;""",
    con=eng
    )
df.head()



,windfarm_id,latitude,longitude
0,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7218,4.46681
1,1ec6d7a1-4bf4-6ab6-a587-df2d2b033685,48.6599,3.97209
2,1ec6d7a1-4bf8-62b0-ac1e-df2d2b033685,49.4222,4.54974
3,1ec6d7a1-4bfb-6adc-9f35-df2d2b033685,48.6326,4.18423
4,1ec6d7a1-4bff-615a-92af-df2d2b033685,48.6150,4.15148


In [3]:
client = mongodb_connection()

def last_weather_forecast_to_df(client):
    # Get the last update
    last_forcast_update = client.forecast.find({}, {"data" : 1}).sort("extract_date", -1).limit(1)
    last_forcast_update = list(last_forcast_update)[0]["data"]
    print(last_forcast_update)

    # Select required data from the document
    list_data = []
    for windfarm in last_forcast_update:
        for forecast in windfarm["list"]:
            dict_forecast = {}
            for key, value in forecast.items():
                if key == "dt_txt":
                    dict_forecast["forecast_date"] = value
                elif key == "wind":
                    dict_forecast["windspeed"] = value["speed"]
                else :
                    continue
            dict_forecast["windfarm_id"] = windfarm["windfarm_id"]
            list_data.append(dict_forecast)

    # Transform list in to DataFrame
    df = pd.DataFrame(list_data)
    return df

df_forecast = last_weather_forecast_to_df(client)

df_forecast.head()

[{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1686225600, 'main': {'temp': 299.09, 'feels_like': 299.07, 'temp_min': 297.21, 'temp_max': 299.09, 'pressure': 1013, 'sea_level': 1013, 'grnd_level': 993, 'humidity': 51, 'temp_kf': 1.88}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'clouds': {'all': 0}, 'wind': {'speed': 2.9, 'deg': 63, 'gust': 3.03}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': datetime.datetime(2023, 6, 8, 12, 0)}, {'dt': 1686236400, 'main': {'temp': 298.6, 'feels_like': 298.48, 'temp_min': 297.63, 'temp_max': 298.6, 'pressure': 1013, 'sea_level': 1013, 'grnd_level': 991, 'humidity': 49, 'temp_kf': 0.97}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'clouds': {'all': 0}, 'wind': {'speed': 2.86, 'deg': 59, 'gust': 3.01}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': datetime.datetime(2023, 6, 8, 15, 0)}, {'dt': 1686247200, 'main': {'temp': 296.57,

,windspeed,forecast_date,windfarm_id
0,2.90,2023-06-08 12:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685
1,2.86,2023-06-08 15:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685
2,2.47,2023-06-08 18:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685
3,2.70,2023-06-08 21:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685
4,2.96,2023-06-09 00:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685


In [4]:
df_final=df.merge(df_forecast, on='windfarm_id', how='left')
df_final

,windfarm_id,latitude,longitude,windspeed,forecast_date
0,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7218,4.46681,2.90,2023-06-08 12:00:00
1,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7218,4.46681,2.86,2023-06-08 15:00:00
2,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7218,4.46681,2.47,2023-06-08 18:00:00
3,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7218,4.46681,2.70,2023-06-08 21:00:00
4,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7218,4.46681,2.96,2023-06-09 00:00:00
...,...,...,...,...,...
475,1eca9ed4-f2b0-6690-9d53-f3bf1b9edc7c,48.6431,4.05575,3.97,2023-06-12 21:00:00
476,1eca9ed4-f2b0-6690-9d53-f3bf1b9edc7c,48.6431,4.05575,3.84,2023-06-13 00:00:00
477,1eca9ed4-f2b0-6690-9d53-f3bf1b9edc7c,48.6431,4.05575,4.48,2023-06-13 03:00:00
478,1eca9ed4-f2b0-6690-9d53-f3bf1b9edc7c,48.6431,4.05575,4.72,2023-06-13 06:00:00


In [8]:
import matplotlib.dates as mdates
location = df['latitude'].mean(), df['longitude'].mean()
m = folium.Map(location=location,zoom_start=4)
list_farms = df_final['windfarm_id'].unique().tolist()
for f in list_farms:   
    df_farm = df_final[df_final.windfarm_id==f]
    lat = df_farm['latitude'].iloc[0]
    long = df_farm['longitude'].iloc[0]
    df_farm = df_farm[['forecast_date','windspeed']] 
    # Créer votre graphique Matplotlib
    fig, ax = plt.subplots()
    ax.plot(df_farm['forecast_date'], df_farm['windspeed'],'-o', markersize=4)
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    fig.autofmt_xdate() 
    ax.set_xlabel('Date')
    ax.set_ylabel('Windspeed')
    ax.set_title('Weather')

    # Convertir le graphique en une image au format PNG
    canvas = FigureCanvas(fig)
    buffer = BytesIO()
    canvas.print_png(buffer)
    plt.close(fig)

    # Convertir l'image en représentation base64
    image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
    # Créer une balise HTML pour afficher l'image dans la popup
    html = f'<img src="data:image/png;base64,{image_base64}">'


    popup = folium.Popup(html, max_width=800)
    folium.Marker([lat,long],
                  popup=popup,icon=folium.Icon( icon='home', prefix='fa')).add_to(m)
    
m.save('map.html')





    
   